# 导入模块及配置文件

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
DATASET_PATH = "/home/tarena/桌面/Titanic/data"

# 加载数据

In [3]:
def load_train_dataset():
    csv_path = os.path.join(DATASET_PATH, 'train.csv')
    return pd.read_csv(csv_path)

def load_test_dataset():
    csv_path = os.path.join(DATASET_PATH, 'test.csv')
    return pd.read_csv(csv_path)

In [4]:
titanic_train = load_train_dataset()

In [5]:
titanic_test = load_test_dataset()

# 预览数据

In [6]:
titanic_train.head()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S

In [7]:
titanic_train.shape

(891, 12)

In [8]:
titanic_train.describe()

PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.383838    2.308642   29.699118    0.523008   
std     257.353842    0.486592    0.836071   14.526497    1.102743   
min       1.000000    0.000000    1.000000    0.420000    0.000000   
25%     223.500000    0.000000    2.000000   20.125000    0.000000   
50%     446.000000    0.000000    3.000000   28.000000    0.000000   
75%     668.500000    1.000000    3.000000   38.000000    1.000000   
max     891.000000    1.000000    3.000000   80.000000    8.000000   

            Parch        Fare  
count  891.000000  891.000000  
mean     0.381594   32.204208  
std      0.806057   49.693429  
min      0.000000    0.000000  
25%      0.000000    7.910400  
50%      0.000000   14.454200  
75%      0.000000   31.000000  
max      6.000000  512.329200

In [9]:
titanic_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [10]:
%matplotlib inline
import matplotlib.pyplot as plt
titanic_train.hist(bins=50, figsize=(20, 15))
plt.show()

In [11]:
titanic_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

# 数据整理
## 数据转换

In [12]:
# Add a column to represent if a passenger had a cabin or not
titanic_train['CabinBool'] = titanic_train['Cabin'].apply(lambda x: 1 if not pd.isnull(x) else 0)
titanic_test['CabinBool'] = titanic_test['Cabin'].apply(lambda x: 1 if not pd.isnull(x) else 0)

In [13]:
# Drop column "Cabin" because it's not useful anymore
titanic_train = titanic_train.drop("Cabin", axis=1)
titanic_test = titanic_test.drop("Cabin", axis=1)

In [14]:
# Drop also Name, Ticket and Embarked as they are not relevant for the anlysis
titanic_train = titanic_train.drop("Name", axis=1)
titanic_train = titanic_train.drop("Ticket", axis=1)
titanic_train = titanic_train.drop("Embarked", axis=1)

titanic_test = titanic_test.drop("Name", axis=1)
titanic_test = titanic_test.drop("Ticket", axis=1)
titanic_test = titanic_test.drop("Embarked", axis=1)

In [15]:
# Encode male/female values to numbers - female -> 0, male -> 1
from sklearn.preprocessing import LabelEncoder

train_sex_cat = titanic_train["Sex"]
train_encoder = LabelEncoder()
train_sex_cat_encoded = train_encoder.fit_transform(train_sex_cat)
titanic_train["SexEncoded"] = train_sex_cat_encoded
titanic_train = titanic_train.drop("Sex", axis=1)

test_sex_cat = titanic_test["Sex"]
test_encoder = LabelEncoder()
test_sex_cat_encoded = test_encoder.fit_transform(test_sex_cat)
titanic_test["SexEncoded"] = test_sex_cat_encoded
titanic_test = titanic_test.drop("Sex", axis=1)

In [16]:
# Substitute all NaN values with their median (in this case only Age column have them)
from sklearn.preprocessing import Imputer

train_imputer = Imputer(strategy="median")
train_imputer.fit(titanic_train)
X = train_imputer.transform(titanic_train)
titanic_train = pd.DataFrame(X, columns=titanic_train.columns)

test_imputer = Imputer(strategy="median")
test_imputer.fit(titanic_test)
X = test_imputer.transform(titanic_test)
titanic_test = pd.DataFrame(X, columns=titanic_test.columns)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


## 数据分割

In [17]:
# Split the training dataset in 80% / 20%
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(titanic_train, test_size=0.2, random_state=42)

In [18]:
# Put PassengerId column aside and remove it from original dataset
train_set_passengers_ids = train_set["PassengerId"].copy()
train_set = train_set.drop("PassengerId", axis=1)

test_set_passengers_ids = test_set["PassengerId"].copy()
test_set = test_set.drop("PassengerId", axis=1)

In [19]:

# Separate labels from the rest of the dataset
train_set_labels = train_set["Survived"].copy()
train_set = train_set.drop("Survived", axis=1)

test_set_labels = test_set["Survived"].copy()
test_set = test_set.drop("Survived", axis=1)

In [20]:
# Apply a scaler
from sklearn.preprocessing import MinMaxScaler as Scaler

train_scaler = Scaler()
train_scaler.fit(train_set)
train_set_scaled = train_scaler.transform(train_set)

test_scaler = Scaler()
test_scaler.fit(test_set)
test_set_scaled = test_scaler.transform(test_set)

# 选择并训练模型
### Linear Regression

In [21]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(train_set_scaled, train_set_labels)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [22]:
# Test the score of the training
lin_reg.score(train_set_scaled, train_set_labels)

0.3851886771012908

In [23]:
# Test the score of the model using the test set
lin_reg.score(test_set_scaled, test_set_labels)

0.4470071819384521

### Decision Tree Classifier

In [24]:
from sklearn.tree import DecisionTreeClassifier

tree_cls = DecisionTreeClassifier()
tree_cls.fit(train_set_scaled, train_set_labels)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [25]:
# Test the score of the training
tree_cls.score(train_set_scaled, train_set_labels)

0.9831460674157303

In [26]:
# Test the score of the model using the test set
tree_cls.score(test_set_scaled, test_set_labels)

0.664804469273743

### Random Forest Classifier

In [27]:
from sklearn.ensemble import RandomForestClassifier

rf_cls = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=6, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
rf_cls.fit(train_set_scaled, train_set_labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features=6, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [28]:
# Test the score of the training
rf_cls.score(train_set_scaled, train_set_labels)

0.976123595505618

In [29]:
# Test the score of the model using the test set
rf_cls.score(test_set_scaled, test_set_labels)

0.7988826815642458

#### Grid Search: Random Forest Classifier

In [30]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 7]},
    {'bootstrap': [False], 'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 7]}
]

forest_cls = RandomForestClassifier()

grid_search = GridSearchCV(forest_cls, param_grid, cv=5, scoring='neg_mean_squared_error')

grid_search.fit(train_set_scaled, train_set_labels)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid

In [31]:
grid_search.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features=4, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=30,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

### Linear SVC

In [32]:
from sklearn.svm import LinearSVC

lin_svc = LinearSVC(C=100.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)
lin_svc.fit(train_set_scaled, train_set_labels)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=100.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [33]:
# Test the score of the training
lin_svc.score(train_set_scaled, train_set_labels)

0.8019662921348315

In [34]:
# Test the score of the model using the test set
lin_svc.score(test_set_scaled, test_set_labels)

0.8100558659217877

#### Grid Search: Linear SVC

In [35]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'loss': ['hinge', 'squared_hinge'], 'C': [1.0, 10.0, 100.0]}
]

linear_svc_cls = LinearSVC()

grid_search = GridSearchCV(linear_svc_cls, param_grid, cv=5, scoring='neg_mean_squared_error')

grid_search.fit(train_set_scaled, train_set_labels)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LinearSVC(C=1.0, class_weight=None, dual=True,
                                 fit_intercept=True, intercept_scaling=1,
                                 loss='squared_hinge', max_iter=1000,
                                 multi_class='ovr', penalty='l2',
                                 random_state=None, tol=0.0001, verbose=0),
             iid='warn', n_jobs=None,
             param_grid=[{'C': [1.0, 10.0, 100.0],
                          'loss': ['hinge', 'squared_hinge']}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=0)

In [36]:
grid_search.best_estimator_

LinearSVC(C=100.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

# 将选择的模型应用到训练集,并准备提交

In [37]:
# Prepare the Verify set (the one to predict)
verify_set_passengers_ids = titanic_test["PassengerId"].copy()
titanic_test = titanic_test.drop("PassengerId", axis=1)

In [38]:
# Apply StandardScaler
verify_scaler = Scaler()
verify_scaler.fit(titanic_test)
titanic_test_scaled = verify_scaler.transform(titanic_test)

In [39]:
# Make the prevision
y_pred = lin_svc.predict(titanic_test_scaled)

In [40]:
# Prepare the submission CSV
submission = pd.DataFrame({
        "PassengerId": verify_set_passengers_ids,
        "Survived": y_pred
    })

submission["PassengerId"] = submission["PassengerId"].apply(lambda x: int(x))
submission["Survived"] = submission["Survived"].apply(lambda x: int(x))

submission.to_csv('Titanic_lsvm.csv', index=False)